In [3]:
# %load datafile.py
# @File    : datafile.py
# @AUTH    : swxs
# @Time    : 2018/7/11 15:44

import os
import sys
import math
import time
import datetime
import numpy as np
import pandas as pd
from functools import partial

BASE_URL = os.path.abspath(os.curdir)
BASE_PATH, CURRENT_DIR = os.path.split(BASE_URL)
BASE_DATA_URL = os.path.join(BASE_URL, "data")

get_file_path = partial(os.path.join, BASE_DATA_URL)

options = {
    'display': {
        'max_columns': None,
        'max_colwidth': 25,
        'expand_frame_repr': False,  # Don't wrap to multiple pages
        'max_rows': None,
        'max_seq_items': 50,  # Max length of printed sequence
        'precision': 4,
        'show_dimensions': False
    },
    'mode': {
        'chained_assignment': None  # Controls SettingWithCopyWarning
    }
}

for category, option in options.items():
    for op, value in option.items():
        pd.set_option('{category}.{op}'.format(category=category, op=op), value)


def _get_hdf_keys(fname):
    with pd.HDFStore(os.path.join("data", fname)) as hdf:
        key_list = hdf.keys()
    return key_list


def get_dataframe(fname, key_list):
    df_list = [get_dataframe_by_file(fname, key=x) for x in key_list]
    df = pd.concat(df_list, ignore_index=True)
    return df


def save_dataframe_by_file(abs_filename, df, key="table"):
    ext = os.path.splitext(abs_filename)[1].lower()
    if ext == '.csv':
        df.to_csv(abs_filename, encoding='gb18030', index=False)
    elif ext in ['.xlsx', '.xls']:
        writer = pd.ExcelWriter(abs_filename)
        df.to_excel(writer, "Sheet1", index=False)
        writer.save()
    elif ext in ['.h5']:
        if key == "table":
            df.to_hdf(abs_filename, key, mode="w")
        else:
            df.to_hdf(abs_filename, key)
    else:
        raise Exception('not supported yet')


def get_dataframe_by_file(abs_filename, key="table", **kwargs):
    ext = os.path.splitext(abs_filename)[1].lower()
    if ext == '.csv':
        try:
            df = pd.read_csv(abs_filename, encoding='utf8', **kwargs)
        except UnicodeDecodeError:
            df = pd.read_csv(abs_filename, encoding='gb18030', **kwargs)
    elif ext in ['.xlsx', '.xls']:
        df = pd.read_excel(abs_filename, **kwargs)
    elif ext in ['.h5']:
        try:
            df = pd.read_hdf(abs_filename, key=key)
        except KeyError:
            try:
                df = get_dataframe(abs_filename, _get_hdf_keys(abs_filename))
            except:
                df = pd.DataFrame()
    else:
        raise Exception('not supported yet')
    return df


def save_head_part(df, filename="output.csv"):
    abs_filename = os.path.join(BASE_DATA_URL, filename)
    save_dataframe_by_file(abs_filename, df.head())


def save_all_part(df, filename="output_all.csv"):
    abs_filename = os.path.join(BASE_DATA_URL, filename)
    save_dataframe_by_file(abs_filename, df)


In [11]:
path = get_file_path("2019_09_12_17_19_25_城市信息.xlsx")

In [13]:
df = get_dataframe_by_file(path)
df

,ID,城市,统计数值
0,1,北京,6550034
1,2,天津,634334
2,3,上海,2787768
3,4,广州,674100
4,5,浙江,44260
5,6,宁夏,5350
6,7,江苏,72190
7,8,江西,4360
8,9,湖南,5489
9,10,湖北,34555


                df[
                   df[column.classify_column_id].isin(classify.custom_attr.get("contain", [])),
                   column.col
                ] = classify.name

In [34]:
df.loc[:, "project"] = df["城市"]
df.loc[df["城市"].isin(["山东", "陕西"]), "project"] = "分组1"
df

,ID,城市,统计数值,project,condition
0,1,北京,6550034,北京,北京
1,2,天津,634334,天津,天津
2,3,上海,2787768,上海,上海
3,4,广州,674100,广州,广州
4,5,浙江,44260,浙江,浙江
5,6,宁夏,5350,宁夏,宁夏
6,7,江苏,72190,江苏,江苏
7,8,江西,4360,江西,江西
8,9,湖南,5489,湖南,湖南
9,10,湖北,34555,湖北,湖北


In [35]:
df["城市"].isin(["山东", "陕西"])

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12     True
13    False
14    False
15     True
16    False
17    False
18    False
19    False
Name: 城市, dtype: bool

设置筛选组

In [69]:
cond_1 = (df["城市"] == "山东")
cond_2 = (df["城市"] != "山东")
cond_3 = (df["城市"].str.contains("西"))
cond_4 = (~df["城市"].str.contains("西"))
cond_df = pd.DataFrame([cond_1, cond_2, cond_3, cond_4], index=["cond_1", "cond_2", "cond_3", "cond_4"]).T
cond_df

,cond_1,cond_2,cond_3,cond_4
0,False,True,False,True
1,False,True,False,True
2,False,True,False,True
3,False,True,False,True
4,False,True,False,True
5,False,True,False,True
6,False,True,False,True
7,False,True,True,False
8,False,True,False,True
9,False,True,False,True


In [99]:
check_list = [
    (df["城市"] == "山东"),
    (df["城市"] != "陕西"),
    (~df["城市"].str.contains("西"))
]

total_check_df = None
while check_list:
    try:
        check_df = check_list.pop()
        if isinstance(total_check_df, (pd.DataFrame, pd.Series)):
            total_check_df = total_check_df | check_df
        else:
            total_check_df = check_df
        print(total_check_df)
    except Exception as e:
        print(e)

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7     False
8      True
9      True
10     True
11     True
12     True
13     True
14    False
15    False
16     True
17     True
18     True
19     True
Name: 城市, dtype: bool
0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15    False
16     True
17     True
18     True
19     True
Name: 城市, dtype: bool
0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15    False
16     True
17     True
18     True
19     True
Name: 城市, dtype: bool


In [97]:
check_list = [
    (df["城市"] == "山东"),
    (df["城市"] != "陕西"),
    (~df["城市"].str.contains("西"))
]

total_check_df = None
while check_list:
    try:
        check_df = check_list.pop()
        if isinstance(total_check_df, (pd.DataFrame, pd.Series)):
            total_check_df = total_check_df & check_df
        else:
            total_check_df = check_df
        print(total_check_df)
    except:
        pass

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7     False
8      True
9      True
10     True
11     True
12     True
13     True
14    False
15    False
16     True
17     True
18     True
19     True
Name: 城市, dtype: bool
0      True
1      True
2      True
3      True
4      True
5      True
6      True
7     False
8      True
9      True
10     True
11     True
12     True
13     True
14    False
15    False
16     True
17     True
18     True
19     True
Name: 城市, dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12     True
13    False
14    False
15    False
16    False
17    False
18    False
19    False
Name: 城市, dtype: bool


In [100]:
df.loc[:, "condition"] = df["城市"]
df.loc[total_check_df, "condition"] = "分组1"
df

,ID,城市,统计数值,project,condition
0,1,北京,6550034,北京,分组1
1,2,天津,634334,天津,分组1
2,3,上海,2787768,上海,分组1
3,4,广州,674100,广州,分组1
4,5,浙江,44260,浙江,分组1
5,6,宁夏,5350,宁夏,分组1
6,7,江苏,72190,江苏,分组1
7,8,江西,4360,江西,分组1
8,9,湖南,5489,湖南,分组1
9,10,湖北,34555,湖北,分组1


In [140]:
df.loc[:, "number_step"] = df["统计数值"]

In [141]:
nmin = 0
nmax = 100000
step = 30000

rl = []

current = nmin
while current < nmax:
    if current + step >= nmax:
        rl.append((current, nmax))
    else:
        rl.append((current, current + step))
    current = current + step

df.loc[(rl[-1][0] <= df["统计数值"]) & ( df["统计数值"]<= rl[-1][1]), "number_step"] = f"[{rl[-1][0]}, {rl[-1][1]}]"
for r in rl[-2::-1]:
    df.loc[(r[0] <= df["统计数值"]) & (df["统计数值"] < r[1]), "number_step"] = f"[{r[0]}, {r[1]})"
    
df

,ID,城市,统计数值,project,condition,number_step,number_diy_step
0,1,北京,6550034,北京,分组1,6550034,6550034
1,2,天津,634334,天津,分组1,634334,634334
2,3,上海,2787768,上海,分组1,2787768,2787768
3,4,广州,674100,广州,分组1,674100,674100
4,5,浙江,44260,浙江,分组1,"[30000, 60000)",44260
5,6,宁夏,5350,宁夏,分组1,"[0, 30000)",5350
6,7,江苏,72190,江苏,分组1,"[60000, 90000)",72190
7,8,江西,4360,江西,分组1,"[0, 30000)",4360
8,9,湖南,5489,湖南,分组1,"[0, 30000)",5489
9,10,湖北,34555,湖北,分组1,"[30000, 60000)",34555


In [142]:
df.loc[:, "number_diy_step"] = df["统计数值"]
df.head()

,ID,城市,统计数值,project,condition,number_step,number_diy_step
0,1,北京,6550034,北京,分组1,6550034,6550034
1,2,天津,634334,天津,分组1,634334,634334
2,3,上海,2787768,上海,分组1,2787768,2787768
3,4,广州,674100,广州,分组1,674100,674100
4,5,浙江,44260,浙江,分组1,"[30000, 60000)",44260


In [143]:
path = get_file_path("name.csv")
df = get_dataframe_by_file(path)
df.head()

,key,text,time
0,1,【首页搜索霸王餐VIP免费体验】良品铺子（...,2018-08-30
1,2,在苏州中心的良品铺面积不是很大，被门口的可...,2019-07-25
2,3,良品铺子，很流行的一个零食店铺，类似来伊份...,2019-06-23
3,4,【首页搜索霸王餐VIP免费】苏州中心的这家...,2019-04-11
4,5,他家的烤面筋，，非常辣，口味特别好，对于爱...,2018-06-08


In [146]:
df["time"]=pd.to_datetime(df["time"])
df.head()

,key,text,time
0,1,【首页搜索霸王餐VIP免费体验】良品铺子（...,2018-08-30
1,2,在苏州中心的良品铺面积不是很大，被门口的可...,2019-07-25
2,3,良品铺子，很流行的一个零食店铺，类似来伊份...,2019-06-23
3,4,【首页搜索霸王餐VIP免费】苏州中心的这家...,2019-04-11
4,5,他家的烤面筋，，非常辣，口味特别好，对于爱...,2018-06-08


In [152]:
step_list = [
    {"start": datetime.datetime(2018, 1, 1), "end": datetime.datetime(2018, 6, 1)},
    {"start": datetime.datetime(2018, 6, 1), "end": datetime.datetime(2019, 1, 1)},
    {"start": datetime.datetime(2019, 1, 1), "end": datetime.datetime(2019, 6, 1)}
]
for step in step_list[::-1]:
    start = step.get("start")
    end = step.get("end")
    df.loc[(start <= df["time"]) & (df["time"] <= end), "time_diy_step"] = f"[{start:%Y/%m/%d}, {end:%Y/%m/%d}]"
df

,key,text,time,time_diy_step
0,1,【首页搜索霸王餐VIP免费体验】良品铺子（...,2018-08-30,"[2018/06/01, 2019/01/01]"
1,2,在苏州中心的良品铺面积不是很大，被门口的可...,2019-07-25,NaN
2,3,良品铺子，很流行的一个零食店铺，类似来伊份...,2019-06-23,NaN
3,4,【首页搜索霸王餐VIP免费】苏州中心的这家...,2019-04-11,"[2019/01/01, 2019/06/01]"
4,5,他家的烤面筋，，非常辣，口味特别好，对于爱...,2018-06-08,"[2018/06/01, 2019/01/01]"
5,6,苏州中心的良品铺子中各类瓜子，坚果，奶咯品...,2019-02-09,"[2019/01/01, 2019/06/01]"
6,7,【首页搜索霸王餐VIP免费】关注我赠你V橙...,2019-08-27,NaN
7,8,在苏州中心负一层买了山崎面包，正好坐着休息...,2018-07-14,"[2018/06/01, 2019/01/01]"
8,9,朋友年前给了这家店的礼品卡差点都忘记去领了...,2019-08-16,NaN
9,10,不好吃，坑货比较多，包装倒是比较好看,2019-03-02,"[2019/01/01, 2019/06/01]"


In [155]:
start_time = datetime.datetime.strptime("2019/01/01", "%Y/%m/%d")
start_time

datetime.datetime(2019, 1, 1, 0, 0)

In [159]:
df.loc[:, "time_step"] = df["time"]

In [164]:
df.loc[df["time_step"] > start_time, "time_step"]

1      2019-07-25
2      2019-06-23
3      2019-04-11
5      2019-02-09
6      2019-08-27
8      2019-08-16
9      2019-03-02
12     2019-09-17
13     2019-03-26
16     2019-01-05
17     2019-01-26
19     2019-08-31
24     2019-08-15
25     2019-09-20
26     2019-03-04
33     2019-07-22
35     2019-05-22
37     2019-11-09
45     2019-01-08
49     2019-06-28
52     2019-09-21
55     2019-10-26
56     2019-09-08
59     2019-03-13
61     2019-11-03
67     2019-05-22
72     2019-08-11
74     2019-03-31
76     2019-04-10
77     2019-12-26
79     2019-05-24
80     2019-07-12
82     2019-07-12
83     2019-06-29
84     2019-06-18
89     2019-06-06
91     2019-09-01
95     2019-07-07
96     2019-03-04
99     2019-05-08
103    2019-12-27
108    2019-03-26
109    2019-05-31
111    2019-05-04
113    2019-08-23
114    2019-12-10
118    2019-08-03
121    2019-06-08
122    2019-05-17
123    2019-01-08
125    2019-11-06
131    2019-01-18
132    2019-07-03
133    2019-05-30
135    2019-07-20
136    201

In [165]:
import arrow

In [175]:
arrow.get("2019/01/31").shift(quarters=1)

<Arrow [2019-04-30T00:00:00+00:00]>

In [246]:
arrow.utcnow().shift(days=4).strftime("%Y-%m-%d %w %W")

'2019-12-08 0 48'

In [249]:
day = arrow.get("2019/12/08")

day.shift(days=-day.weekday()).shift(weeks=-1).strftime("%Y-%m-%d %U")

'2019-11-25 47'

In [261]:
day = arrow.get("2019/12/08")

day.replace(month=(((day.month + 2) // 3 - 1) * 3 + 1), day=1).shift(quarters=0).strftime("%Y-%m-%d %U")

'2019-10-01 39'

In [262]:
dir(arrow.get(arrow.utcnow()).date())

['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rsub__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 'ctime',
 'day',
 'fromordinal',
 'fromtimestamp',
 'isocalendar',
 'isoformat',
 'isoweekday',
 'max',
 'min',
 'month',
 'replace',
 'resolution',
 'strftime',
 'timetuple',
 'today',
 'toordinal',
 'weekday',
 'year']

In [270]:
rng = [datetime.date.today(), datetime.date(2019, 12, 1)]
f"[{rng[0]:%Y}年Q{arrow.get(rng[0]).quarter}, {rng[1]:%Y}年Q{arrow.get(rng[0]).quarter}]"

'[2019年Q4, 2019年Q4]'

为空， 非空

In [271]:
df = pd.DataFrame({"a": ["", np.nan, "a"]})

In [272]:
df

,a
0,
1,NaN
2,a


In [273]:
df[df["a"].isnull()]

,a
1,NaN


In [274]:
df[df["a"].notnull()]

,a
0,
2,a
